In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


2024-10-08 18:59:22.122710: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Generate synthetic stock price data for demonstration
np.random.seed(42)
data = np.sin(np.arange(0, 200, 0.1)) + np.random.normal(0, 0.1, 2000) # Synthetic data
data = data.reshape(-1, 1)


In [ ]:
# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)


In [ ]:
# Prepare sequences of 60 timesteps to predict the next timestep
def create_sequences(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_sequences(data_scaled, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)


In [ ]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:
# Build RNN (LSTM) model
model = models.Sequential()
model.add(layers.LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(layers.LSTM(50))
model.add(layers.Dense(1))


In [ ]:
# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Callback to track learning rate
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Learning Rate: {self.model.optimizer.lr.numpy()}")


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [ ]:
# Predict and reverse the scaling
predicted_stock_price = model.predict(X_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price.reshape(-1, 1))


In [ ]:
# Visualize results
plt.figure(figsize=(12, 4))


In [ ]:
# Training Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('RNN Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()


In [ ]:
# Plot predictions vs actual values
plt.subplot(1, 2, 2)
plt.plot(scaler.inverse_transform(y_test.reshape(-1, 1)), label='True Price')
plt.plot(predicted_stock_price, label='Predicted Price')
plt.title('RNN Stock Price Prediction')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.legend()

plt.show()
